In [ ]:
# default_exp learner

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

from nbdev import *
from nbdev.export import notebook2script

In [ ]:
# export

import math
# import logging

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR, OneCycleLR
from torch.utils.data.dataloader import DataLoader

from tqdm import tqdm
from pathlib import Path

# Learner

## Utils

In [ ]:
# export
class AverageMeter:
    
    def __init__(self, store_vals=False, store_avgs=False):
        self.store_vals = store_vals
        self.store_avgs = store_avgs
        if store_vals: self.values = []
        if store_avgs: self.avgs = []
        self.sum, self.n, self.avg = 0, 0, None
        
    def update(self, v):
        if self.store_vals: self.values.append(v)
        self.n += 1
        self.sum += v
        self.avg = self.sum/self.n
        
    def reset(self):
        if self.store_avgs and self.avg: self.avgs.append(self.avg)
        self.sum, self.n, self.avg = 0, 0, None

In [ ]:
# export
def accuracy_binary(pred, targ):
    return ((pred>0).float() == targ).float().mean()

## Callbacks

In [ ]:
# export
class Callback:
    def __init__(self, learn):
        self.learn = learn
        
    def __getattr__(self, attr):
        pass

In [ ]:
# hide
class TrainEvalCallback(Callback):
    
    def before_train(self):
        self.learn.model.train()
        self.learn.training = True
    
    def before_validate(self):
        self.learn.model.eval()
        self.learn.training = False

In [ ]:
# hide
class SaveModelCallback(Callback):
    pass

## Learner

In [ ]:
data_train = np.random.normal(size=(16*100, 10)).astype(np.float32)
data_valid = np.random.normal(size=(16*100, 10)).astype(np.float32)

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class DS(Dataset):
    def __init__(self, data):
        self.x = data
        self.y = data.sum(axis=1, keepdims=True)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return len(self.x)

In [ ]:
train_dl = DataLoader(DS(data_train), 16)
valid_dl = DataLoader(DS(data_valid), 16)
dls = [train_dl, valid_dl]

In [ ]:
x, y = next(iter(train_dl))

In [ ]:
model = nn.Sequential(nn.Linear(10, 10),
                      nn.BatchNorm1d(10),
                      nn.ReLU(),
                      nn.Linear(10, 1))

In [ ]:
pred = model(x)
pred.shape, y.shape

(torch.Size([16, 1]), torch.Size([16, 1]))

In [ ]:
nn.BCEWithLogitsLoss()(pred, y)

tensor(0.7175, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [ ]:
def wd_param_groups(model:nn.Module, wd:float):
    decay, no_decay = [], []
    for n, c in model.named_children():
        if n == 'stem':
            
            no_decay += [p for p in c.parameters() if p.requires_grad]
        else:
            no_decay += [p for p in c.parameters() if (p.requires_grad and len(p.shape)==1)]
            decay += [p for p in c.parameters() if p.requires_grad and len(p.shape)!=1]
    params = [
            {"params": decay, "weight_decay": wd},
            {"params": no_decay, "weight_decay": 0.0},
    ]
    return params

In [ ]:
from nn4tab.model import TabularModel

In [ ]:
tabnn = TabularModel([10,10], ((5,5), (4,4)), 2, 1)

In [ ]:
tabnn

TabularModel(
  (stem): TabInputBlock(
    (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (embs): ModuleList(
      (0): Embedding(5, 5)
      (1): Embedding(4, 4)
    )
  )
  (lins): Sequential(
    (0): LinearBlock(
      (m): Sequential(
        (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Linear(in_features=11, out_features=10, bias=True)
        (2): ReLU()
      )
    )
    (1): LinearBlock(
      (m): Sequential(
        (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Linear(in_features=10, out_features=10, bias=True)
        (2): ReLU()
      )
    )
    (2): LinearBlock(
      (m): Sequential(
        (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Linear(in_features=10, out_features=1, bias=True)
      )
    )
  )
)

In [ ]:
wd_param_groups(tabnn, 1.)

[{'params': [Parameter containing:
   tensor([[ 2.7277e-01, -2.2471e-01,  2.6523e-01,  6.0695e-02, -2.5695e-01,
             9.6278e-02,  1.2022e-01, -1.7679e-01, -1.6692e-01,  2.2691e-01,
            -2.2678e-01],
           [ 5.4562e-02,  2.7151e-01,  1.9484e-01, -1.3532e-01, -1.0771e-01,
            -2.6080e-02, -1.0610e-01,  1.2190e-01, -2.8229e-04,  2.8142e-01,
            -1.1737e-01],
           [-4.7730e-02, -2.9663e-01,  1.9810e-01,  2.2263e-01,  1.9223e-01,
             1.6017e-01,  1.0125e-01, -1.9813e-01, -5.7727e-02, -9.3859e-02,
            -9.6990e-02],
           [ 1.8757e-01, -2.6832e-01,  2.9684e-01, -1.5653e-01, -1.3973e-01,
            -2.1311e-01, -1.3471e-01,  1.5990e-02, -1.3583e-01,  1.6816e-01,
             1.7504e-01],
           [-6.1790e-02, -2.0111e-01,  7.7752e-03, -1.6553e-01,  8.6983e-02,
             2.0088e-01,  4.2001e-02, -8.5758e-03,  2.3476e-02,  2.3230e-02,
             2.1135e-01],
           [ 1.1087e-01, -2.8298e-01, -6.6622e-02,  1.8854e-02, -

#### Learner V0

In [ ]:
# export
class LearnerV0:
    
    def __init__(self, model, dataloaders, opt_func, loss_func, metrics=None, use_gpu=True, savepath='./models'):
        
        self.device = 'cuda' if (torch.cuda.is_available() and use_gpu) else 'cpu'
        self.model = model.to(self.device)
        
        self.train_dl = dataloaders[0]
        self.valid_dl = dataloaders[1]
        self.test_dl = dataloaders[2] if len(dataloaders)>2 else None
        
        self.opt_func = opt_func
        self.loss_func = loss_func
        self.metrics = metrics
        
        self.train_losses = AverageMeter(store_vals=True)
        self.valid_losses = AverageMeter(store_avgs=True)
        self.accs = AverageMeter()
#         self.optimizer = opt_func([p for p in self.model.parameters() if p.requires_grad])
        
        self.savepath = Path(savepath)
        if not self.savepath.exists():
            self.savepath.mkdir()
        self.training = True
        self.epoch = -1
        
    def fit(self, epochs, lr=1e-2):
        
        self.optimizer = self.opt_func([p for p in self.model.parameters() if p.requires_grad], lr)
        
        for e in range(epochs):
            self.epoch += 1
            train_loss = self.train()
            self.train_losses.reset()
            
            valid_loss, acc = self.validate()
            self.valid_losses.reset()
            self.accs.reset()
            
#             print('Train loss = {:f}; valid loss = {:f}; {} = {:f}'.\
#                   format(train_loss, valid_loss, self.metrics.__name__, acc))
            self.save_model()
        
    
    def train(self):
        
        self.model.train()
        pbar = tqdm(self.train_dl)
        for x_cat, x_cont, y in pbar:
            x_cat = x_cat.to(self.device, dtype=torch.long)
            x_cont = x_cont.to(self.device)
            y = y.to(self.device)

            self.optimizer.zero_grad()
            pred = self.model(x_cat, x_cont)
            loss = self.loss_func(pred, y)

            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), grad_norm_clip)
            self.optimizer.step()

            self.train_losses.update(loss.item())
            pbar.set_description(f'epoch {self.epoch+1}: train loss {self.train_losses.avg:.4f}')
        return self.train_losses.avg
    
    def validate(self):
        
        self.model.eval()
        pbar = tqdm(self.valid_dl)
        for x_cat, x_cont, y in pbar:
            x_cat = x_cat.to(self.device, dtype=torch.long)
            x_cont = x_cont.to(self.device)
            y = y.to(self.device)

            with torch.no_grad():
                pred = self.model(x_cat, x_cont)
                loss = self.loss_func(pred, y)

            self.valid_losses.update(loss.item())
            self.accs.update(accuracy_binary(pred, y).item())
            pbar.set_description(f'epoch {self.epoch+1}: valid loss {self.valid_losses.avg:.4f}, accuracy {self.accs.avg :.4f}')
        
        return self.valid_losses.avg, self.accs.avg
    
    def save_model(self, fn='ckpt_', path=None):
        if not path: path = self.savepath
        fn += str(self.epoch) + '.pt'
        torch.save(self.model.state_dict(), path/fn)
    
    def load_model(self, fn, path=None):
        if not path: path = self.savepath
        self.model.load_state_dict(torch.load(path/fn))

#### Learner V1

In [ ]:
# export
class LearnerV1:
    
    def __init__(self, model, dataloaders, opt_func, loss_func, metrics=None, use_gpu=True, savepath='./models'):
        
        self.device = 'cuda' if (torch.cuda.is_available() and use_gpu) else 'cpu'
        self.model = model.to(self.device)
        
        self.train_dl = dataloaders[0]
        self.valid_dl = dataloaders[1]
        self.test_dl = dataloaders[2] if len(dataloaders)>2 else None
        
        self.opt_func = opt_func
        self.loss_func = loss_func
        self.metrics = metrics
        
        self.train_losses = AverageMeter(store_vals=True)
        self.valid_losses = AverageMeter(store_avgs=True)
        self.accs = AverageMeter()
#         self.optimizer = opt_func([p for p in self.model.parameters() if p.requires_grad])
        
        self.savepath = Path(savepath)
        if not self.savepath.exists():
            self.savepath.mkdir()
        self.training = True
        self.epoch = -1
        
    def fit(self, epochs, lr=1e-2, wd=0.):
        
        if wd:
            params = wd_param_groups(self.model, wd)
        else:
            params = [p for p in self.model.parameters() if p.requires_grad]
        self.optimizer = self.opt_func(params, lr)
        
        for e in range(epochs):
            self.epoch += 1
            train_loss = self.train()
            self.train_losses.reset()
            
            valid_loss, acc = self.validate()
            self.valid_losses.reset()
            self.accs.reset()
            
            self.save_model()
        
    
    def train(self):
        
        self.model.train()
        pbar = tqdm(self.train_dl)
        for x_cat, x_cont, y in pbar:
            x_cat = x_cat.to(self.device, dtype=torch.long)
            x_cont = x_cont.to(self.device)
            y = y.to(self.device)

            self.optimizer.zero_grad()
            pred = self.model(x_cat, x_cont)
            loss = self.loss_func(pred, y)

            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), grad_norm_clip)
            self.optimizer.step()

            self.train_losses.update(loss.item())
            pbar.set_description(f'epoch {self.epoch+1}: train loss {self.train_losses.avg:.4f}')
        return self.train_losses.avg
    
    def validate(self):
        
        self.model.eval()
        pbar = tqdm(self.valid_dl)
        for x_cat, x_cont, y in pbar:
            x_cat = x_cat.to(self.device, dtype=torch.long)
            x_cont = x_cont.to(self.device)
            y = y.to(self.device)

            with torch.no_grad():
                pred = self.model(x_cat, x_cont)
                loss = self.loss_func(pred, y)

            self.valid_losses.update(loss.item())
            self.accs.update(accuracy_binary(pred, y).item())
            pbar.set_description(f'epoch {self.epoch+1}: valid loss {self.valid_losses.avg:.4f}, accuracy {self.accs.avg :.4f}')
        
        return self.valid_losses.avg, self.accs.avg
    
    def save_model(self, fn='ckpt_', path=None):
        if not path: path = self.savepath
        fn += str(self.epoch) + '.pt'
        torch.save(self.model.state_dict(), path/fn)
    
    def load_model(self, fn, path=None):
        if not path: path = self.savepath
        self.model.load_state_dict(torch.load(path/fn))

In [ ]:
notebook2script()

Converted 00_core.ipynb.
Converted 00a_test_utils.ipynb.
Converted 01_data.ipynb.
Converted 02_model.ipynb.
Converted 03_learner.ipynb.
Converted index.ipynb.
